This notebook mines for any recent `stock` news sources, and saves them to a file. This is run daily.

In [2]:

import sys
import os 
from finvizfinance.news import News
from finvizfinance.screener.overview import Overview
from finvizfinance.quote import finvizfinance
import pandas as pd 
from datetime import datetime
import time
from config import STOCK_PATH, STOCK_DIR



if not os.path.exists(STOCK_PATH): os.makedirs(STOCK_PATH)




In [ ]:
# Get list of stocks that meet market cap criteria. Since sp500 is self-cleansing, companies get aquired / delisted frequently.

market_cap = {
        'Mega' : 'Mega ($200bln and more)',
        'Large' : '+Large (over $10bln)',
        'Mid' : '+Mid (over $2bln)',
}

foverview = Overview()
filters_dict = {'Market Cap.': market_cap['Large'], 'Country': 'USA'}

foverview.set_filter(filters_dict=filters_dict)
df = foverview.screener_view() 
df['Market Cap (billions)'] = df['Market Cap'] / 1e9
df.drop(columns = ['P/E', 'Change', 'Market Cap', 'Volume'], inplace = True)

df.to_csv('../stocks/stocks.csv', index = False)                               # sink all marketcap+ stocks

In [9]:
df = pd.read_csv('../stocks/stocks.csv')
df.sort_values(by='Market Cap (billions)', ascending= False)

,Ticker,Company,Sector,Industry,Country,Price,Market Cap (billions)
440,NVDA,NVIDIA Corp,Technology,Semiconductors,USA,149.43,3822.53
3,AAPL,Apple Inc,Technology,Consumer Electronics,USA,245.00,3695.93
410,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,USA,427.85,3217.26
276,GOOGL,Alphabet Inc,Communication Services,Internet Content & Information,USA,196.87,2447.62
275,GOOG,Alphabet Inc,Communication Services,Internet Content & Information,USA,197.96,2445.67
...,...,...,...,...,...,...,...
392,MGM,MGM Resorts International,Consumer Cyclical,Resorts & Casinos,USA,33.94,10.23
446,OHI,"Omega Healthcare Investors, Inc",Real Estate,REIT - Healthcare Facilities,USA,37.31,10.19
545,SNX,TD Synnex Corp,Technology,Electronics & Computer Distribution,USA,118.30,10.19
109,CFLT,Confluent Inc,Technology,Software - Infrastructure,USA,29.83,10.13


In [7]:
df = pd.read_csv(f'{STOCK_DIR}/stocks.csv')

def get_news(df : pd.Series):       # sinks new news sources to csv files
    try:
            
        time.sleep(0.2)
        file = f"{STOCK_PATH}/{df['Ticker']}.csv"
        whitelist = ['Bloomberg', 'Reuters']
        news_df = finvizfinance(df['Ticker']).ticker_news()
        news_df = news_df[news_df['Source'].isin(whitelist)]              # whitelist

        last_modified = datetime.fromtimestamp(os.path.getmtime(file))
        news_df = news_df[(news_df['Date'] >= last_modified)].sort_values('Date')       # filter new news inbetween modified date and today

        if not os.path.exists(file):
            news_df.to_csv(file, index = False)  
            print(f'created {file} ...')

        news_df.to_csv(file, mode='a', header=False, index=False)         # append to existing news
        print(f'found {len(news_df)} new sources to {file} ...')   

    except Exception as e:
        raise e

    

df.apply(get_news, axis = 1)            # apply get_news() on every row


# get_news(df.iloc[465])



found 0 new sources to ../stocks/tickers/A.csv ...
found 0 new sources to ../stocks/tickers/AA.csv ...
found 0 new sources to ../stocks/tickers/AAON.csv ...
found 0 new sources to ../stocks/tickers/AAPL.csv ...
found 0 new sources to ../stocks/tickers/ABBV.csv ...
found 0 new sources to ../stocks/tickers/ABNB.csv ...
found 0 new sources to ../stocks/tickers/ABT.csv ...
found 0 new sources to ../stocks/tickers/ACI.csv ...
found 0 new sources to ../stocks/tickers/ACM.csv ...
found 0 new sources to ../stocks/tickers/ADBE.csv ...
found 0 new sources to ../stocks/tickers/ADI.csv ...
found 0 new sources to ../stocks/tickers/ADM.csv ...
found 0 new sources to ../stocks/tickers/ADP.csv ...
found 0 new sources to ../stocks/tickers/ADSK.csv ...
found 0 new sources to ../stocks/tickers/AEE.csv ...
found 0 new sources to ../stocks/tickers/AEP.csv ...
found 0 new sources to ../stocks/tickers/AFG.csv ...
found 0 new sources to ../stocks/tickers/AFL.csv ...
found 0 new sources to ../stocks/tickers/AF

Exception: 404 Client Error: Not Found for url: https://finviz.com/quote.ashx?t=CTLT